In [110]:
import sys
!{sys.executable} -m pip install requests

In [111]:
# GET THE CONFIG
import utils
user1 = utils.getenvcached("QUETZALCOATL_USER1")
token1 = utils.getenvcached("QUETZALCOATL_TOKEN1")

In [112]:
from LaplaceAPIClient import *
import os

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
import random
import math

In [113]:

# Generate some train & test data, using sklearn here but no obligation really
st = random.getstate()
random.seed(42)
w = 3.14*20
features = [
    (math.sin(x*w) + random.uniform(-0.05, 0.05),
    math.cos(x*w) + random.uniform(-0.01, 0.03))
    for x in [
        k * 0.001
        for k in range(1000)
    ]
]
labels = [1 if x+y + random.uniform(-0.01, 0.01) >= 0.7 else 0 for (x,y) in features]
random.setstate(st)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [114]:
xfeatures = [[x for x in i] for i in X_train]
xlabels = y_train
tfeatures = [[x for x in i] for i in X_test]
tlabels = y_test

print(repr(xfeatures))
print(repr(xlabels))
print(repr(tfeatures))
print(repr(tlabels))

tag = "TestOnly#TestML19"
config = {"n_estimators":50,"learning_rate":0.2,"random_state":42,"early_stopping_rounds":10,"eval_metric":"error","model_type":"onnx/xgboost"}

[[0.9351527217972275, -0.2426168453988395], [0.8219714029266099, -0.5657651789790573], [-0.3351337876148167, 0.934193100139524], [-0.44883769580676547, -0.8884235552103906], [0.809962939441616, -0.5961280281021379], [-0.30551463173062704, 0.9545219335146586], [-0.7748416468715825, -0.5913568827687687], [0.8429184746723127, 0.464770135481844], [-0.332485074625537, 0.9400270118940165], [0.9620050741481805, 0.09205845666038276], [-0.8270468688934837, -0.5971420135551074], [0.5530372896416698, -0.8413535119677567], [0.9954574121632492, 0.029750423681943], [0.9965441853218779, -0.08920761222601443], [-0.32108227021836383, -0.9587286108357367], [0.9829978207258676, -0.36356323666772766], [-0.7237256099081595, -0.6327329883956004], [0.06744324051920576, -1.0023922126579843], [0.36940972460127247, 0.9453053881884629], [0.47830753047291563, 0.8803719824180317], [0.318941580297867, 0.9562619858157693], [0.8960874893812611, -0.5104758418833216], [-1.0119532246760923, -0.013742538595084796], [0.11

In [115]:
import requests
import json

url = "https://api2.anzaetek.com:443/execute"

def query19_train(config, tag, xfeatures, xlabels, tfeatures, tlabels):
    train_q = { "__class__":"MLTrainingProblem", 
                "query":{
                    "Config":config,
                    "Info":{"user":user1,"token":token1},"Tag":tag,
                    "Features":xfeatures,"Labels":xlabels,
                    "TestFeatures":tfeatures,"TestLabels":tlabels}}

    train_query = {'user': user1, 'token': token1, 'query': json.dumps(train_q)}

    post_response = requests.post(url = url, json=train_query)
    rv = post_response.json()
    #print(rv)
    return rv['Results']['report']

Train the model

In [116]:
status = query19_train(config, tag, xfeatures, xlabels, tfeatures, tlabels)
print(status)

{'accuracy': 0.985, 'confusion_matrix': [[128, 0], [3, 69]]}


In [117]:
tag = "TestOnly#TestML19"
ifeatures = [[x for x in i] for i in X_test]

In [118]:
import requests
import json

url = "https://api2.anzaetek.com:443/execute"

def query19_infer(tag, ifeatures):
    infer_q = { "__class__":"MLInferenceProblem", 
            "query": {
                "Config": {},
                "Info":{"user":user1,"token":token1},
                "Tag": tag,
                "Features":ifeatures
                }
            }
    infer_query = {'user': user1, 'token': token1, 
                'query': json.dumps(infer_q) }
    post_response = requests.post(url = url, json=infer_query)
    rv = post_response.json()
    #print(rv)
    return rv['Labels']

Run inference (use the model).

In [119]:
ir = query19_infer(tag, ifeatures)
print(ir)

[1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0]
